In [7]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sbn
%matplotlib inline

CHUNKSIZE = 10000
NROWS = 50000
DATA_DIR = "/Users/xiuqi/Downloads/Data/Bosch"
TRAIN_DATE = "{0}/train_date.csv".format(DATA_DIR)
TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)

In [4]:
date_cols = pd.read_csv(TRAIN_DATE,nrows=1)
num_cols = pd.read_csv(TRAIN_NUMERIC,nrows=1)
print(date_cols.columns)
print(num_cols.columns)

Index(['Id', 'L0_S0_D1', 'L0_S0_D3', 'L0_S0_D5', 'L0_S0_D7', 'L0_S0_D9',
       'L0_S0_D11', 'L0_S0_D13', 'L0_S0_D15', 'L0_S0_D17',
       ...
       'L3_S50_D4246', 'L3_S50_D4248', 'L3_S50_D4250', 'L3_S50_D4252',
       'L3_S50_D4254', 'L3_S51_D4255', 'L3_S51_D4257', 'L3_S51_D4259',
       'L3_S51_D4261', 'L3_S51_D4263'],
      dtype='object', length=1157)
Index(['Id', 'L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8',
       'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16',
       ...
       'L3_S50_F4245', 'L3_S50_F4247', 'L3_S50_F4249', 'L3_S50_F4251',
       'L3_S50_F4253', 'L3_S51_F4256', 'L3_S51_F4258', 'L3_S51_F4260',
       'L3_S51_F4262', 'Response'],
      dtype='object', length=970)


In [8]:
date_chunks = pd.read_csv(TRAIN_DATE, index_col=0, chunksize=CHUNKSIZE, dtype=np.float32, nrows = NROWS)
num_chunks = pd.read_csv(TRAIN_NUMERIC, index_col=0, usecols=list(range(1,len(num_cols.columns)-1)),
                         chunksize=CHUNKSIZE, dtype=np.float32, nrows = NROWS)
X = pd.concat([pd.concat([dchunk,nchunk],axis=1).sample(frac=0.05,random_state=14) for dchunk,nchunk in zip(date_chunks,num_chunks)],axis=0)
X.iloc[:10,:]

ValueError: Shape of passed values is (2123, 794522), indices imply (2123, 20000)

In [ ]:
y = pd.read_csv(TRAIN_NUMERIC, index_col=0, usecols=[0,969], dtype=np.float32)
y.iloc[:10,:]
y_val = y.loc[X.index].values.ravel()
X = X.values
clf = XGBClassifier(base_score=0.05)
clf.fit(X,y_val)

In [ ]:
plt.hist(clf.feature_importances_[clf.feature_importances_>0])
important_indices = np.where(clf.feature_importances_>0.005)[0]
print(important_indices)

In [ ]:
date_cols = pd.read_csv(TRAIN_DATE,index_col=0,nrows=1)
n_date_features = len(date_cols.columns)
X = np.concatenate([pd.read_csv(TRAIN_DATE,index_col=0,dtype=np.float32,usecols=np.concatenate([[0],important_indices[important_indices<n_date_features]+1])).values,
               pd.read_csv(TRAIN_NUMERIC,index_col=0,dtype=np.float32,usecols=np.concatenate([[0],important_indices[important_indices>=n_date_features]+1-n_date_features])).values],axis=1)

In [ ]:
y = pd.read_csv(TRAIN_NUMERIC,index_col=0,dtype=np.float32,usecols=[0,969]).values.ravel()

In [ ]:
date_cols = pd.read_csv(TRAIN_DATE,index_col=0,nrows=1)
n_date_features = len(date_cols.columns)
X = np.concatenate([pd.read_csv(TRAIN_DATE,index_col=0,dtype=np.float32,usecols=np.concatenate([[0],important_indices[important_indices<n_date_features]+1])).values,
               pd.read_csv(TRAIN_NUMERIC,index_col=0,dtype=np.float32,usecols=np.concatenate([[0],important_indices[important_indices>=n_date_features]+1-n_date_features])).values],axis=1)

In [ ]:
thresholds = np.linspace(0.01,0.99,50)
mcc = np.array([matthews_corrcoef(y,preds>thr) for thr in thresholds])
plt.plot(thresholds,mcc)
best_threshold = thresholds[mcc.argmax()]
print(mcc.max())